# Prelims

In [2]:
%run pipInstall.ipynb
%run importLib.ipynb
%run getFrame.ipynb

2023-04-17 12:56:12.917460: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-17 12:56:14.893604: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-17 12:56:14.899574: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 12:56:19.997033: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import cv2

In [4]:
def get_gt(videoPath):
    vidcap = cv2.VideoCapture(videoPath)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    
    rgb_orig = load_rgb_video(video_path=videoPath,fps=fps)
    rgb_input = prepare_input(rgb_orig)
    rgb_slides, t_mid, clip_len= sliding_windows(rgb=rgb_input)
    num_clips = rgb_slides.shape[0]
    return rgb_slides, t_mid, num_clips

In [5]:
def to_gif(images, filename='./animation.gif'):
    converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
    imageio.mimsave(filename, converted_images, fps=25)
    return embed.embed_file(filename)

In [6]:
def create_GIF(videoPath, rgb_slides):
    !mkdir Output
    filename = videoPath[:-4]
    for i in range(len(rgb_slides)):
        video = tf.transpose(rgb_slides[i], [1,2,3,0,])
        clip = to_gif(video, "./Output/"+filename+"_"+str(i)+".gif")

In [7]:
videoPath = 'Annotated_Test_Set/finethank.mp4'

In [8]:
rgb_slides, t_mid, num_clips=get_gt(videoPath)

Loaded video Annotated_Test_Set/finethank.mp4 with 132 frames [1080hx1920w] res. at 25.0
117 clips resulted from sliding window processing.


# Reading csv file

In [9]:
import pandas as pd
import math

In [10]:
df = pd.read_csv('Annotated_Test_Set/0_1_I_am_fine.csv')

In [11]:
annotation=df.iloc[1:,2:]
annotation = annotation.reset_index()  # make sure indexes pair with number of rows

In [12]:
annotList = []
for index, row in annotation.iterrows():
    label,start,stop=row[3][13], row[4], row[5]
    annotList.append({"label":label, "start":start, "stop":stop})

In [13]:
frameList=[0 for i in range(int(annotList[-1]["stop"]))]
firstFrame = int(annotList[0]["start"])
lastFrame = int(annotList[-1]["stop"])

In [16]:
frameList = {}
for annotDict in annotList:
    print(annotDict)
    start = int(annotDict["start"])
    stop = int(annotDict["stop"])
    start = math.floor(start)
    stop = math.floor(stop)
    
    if(start == firstFrame):
        start+=1
    if(stop == lastFrame):
        stop+=1
    
    for i in range(start, stop):
        frameList[i] = int(annotDict["label"])


{'label': '1', 'start': 0.0, 'stop': 20.225}
{'label': '0', 'start': 20.225, 'stop': 33.5}
{'label': '1', 'start': 33.5, 'stop': 41.3}
{'label': '0', 'start': 41.3, 'stop': 63.175}
{'label': '1', 'start': 63.175, 'stop': 72.55}
{'label': '0', 'start': 72.55, 'stop': 89.825}
{'label': '1', 'start': 89.825, 'stop': 95.225}
{'label': '0', 'start': 95.225, 'stop': 117.1}
{'label': '1', 'start': 117.1, 'stop': 132.0}


In [17]:
groundTruth = [frameList[int(i)] for i in t_mid]
len(groundTruth)

117

In [18]:
#groundTruth

In [19]:
'''
.04 sec = 1 unit
1 msec = z
40 msec = 1 unit
z = 1/40 = 0.025 units
1 msec = 0.025 units
1 sec = 25 units
x:y sec = 25x: 25 y
'''





'\n.04 sec = 1 unit\n1 msec = z\n40 msec = 1 unit\nz = 1/40 = 0.025 units\n1 msec = 0.025 units\n1 sec = 25 units\nx:y sec = 25x: 25 y\n'

In [20]:
import pickle
pred = pickle.load(open("Annotated_Test_Set/predictions.pkl", "rb"))

In [21]:
print(pred)

[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [22]:
len(pred)

117

In [24]:
from eval import Metric 

In [25]:
metric = Metric('train')

In [26]:
metric.calc_scores_per_batch(torch.Tensor([pred]), torch.Tensor([groundTruth]), torch.Tensor([groundTruth]))

In [27]:
metric.calc_metrics()

In [28]:
metric.mean_f1b

54.17